# Avaliando a qualidade do modelo via cross validation
Esse notebook faz parte do material de apoio do tutorial [Introdução ao Scikit-learn - Parte 3: avaliando o modelo via cross validation](http://computacaointeligente.com.br/outros/intro-sklearn-part-3/)

In [5]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import numpy as np

## Carregando os dados 

In [6]:
wine_dataset = load_wine()
X = wine_dataset['data']
y = wine_dataset['target']
nome_das_classes = wine_dataset.target_names

## Criando o pipeline e executando N vezes
Antes de executarmos o *cross-validation* vamos executar o modelos `N` vezes. Mas dessa vez, sempre antes de treinar o modelo, vamos embaralhar a base (perceba que não setamos o parâmetro `random_state`. Por default ele fica como `None`).

Nesse experimento, calculamos a média da acurácia. Fica como exercício fazer o mesmo para a matriz de confusão, que foi calculada na parte 2.

In [21]:
K = 3
N = 10
acuracias = list()

matrizes_conf = list()

knn_pipeline = Pipeline(steps=[
  ("normalizacao", MinMaxScaler()),  
  ("KNN", KNeighborsClassifier(n_neighbors=K))
])

for i in range(N):
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.3, shuffle=True)
    knn_pipeline.fit(X_treino, y_treino)
    ac_i = knn_pipeline.score(X_teste, y_teste)
    acuracias.append(ac_i)

    y_pred = knn_pipeline.predict(X_teste)
    mat_conf_i = confusion_matrix(y_teste, y_pred)
    matrizes_conf.append(mat_conf_i)

print("- Acurácia:")
print(f"Media: {round(np.mean(acuracias) * 100, 2)}%")
print(f"Desvio padrão: {round(np.std(acuracias) * 100, 2)}%")

matrizes_conf_media = np.zeros_like(matrizes_conf[0], dtype=float)
for matriz in matrizes_conf:
    matrizes_conf_media += matriz / len(matrizes_conf)

print("\nMatriz de confusão média:")
print(f"Media:\n{matrizes_conf_media}")

- Acurácia:
Media: 95.56%
Desvio padrão: 2.37%

Matriz de confusão média:
Media:
[[18.6  0.   0. ]
 [ 0.7 18.6  1.4]
 [ 0.   0.3 14.4]]


## Utilizando cross-validation
Ok, agora vamos utilizar cross-validation. Mas primeiro, precisamos do modelo base.

In [22]:
knn_pipeline = Pipeline(steps=[
  ("normalizacao", MinMaxScaler()),  
  ("KNN", KNeighborsClassifier(n_neighbors=3))
])

### Usando `cross_val_score`
Essa função básicamente executa o cross-validation de acordo com o número de *folders* informada via o parâmetro `cv`. Na sequência ela retorna uma métrica de performance para cada folder de teste. Por padrão, essa métrica é a acurácia (para classificação). Mas podemos definir usando o parâmetro `scoring`

In [23]:
acuracias = cross_val_score(knn_pipeline, X, y, cv=5)
print("acuracias:", acuracias)
print("acuracia final:", np.mean(acuracias), "+-", np.std(acuracias))

acuracias: [0.91666667 0.94444444 1.         1.         0.91428571]
acuracia final: 0.9550793650793651 +- 0.03817980032602645


In [25]:
f1s = cross_val_score(knn_pipeline, X, y, cv=5, scoring="f1_macro")
print("F1-macro:", f1s)
print("F1-macro:", np.mean(f1s), "+-", np.std(f1s))

F1-macro: [0.9162963  0.94515263 1.         1.         0.91764133]
F1-macro: 0.9558180493969968 +- 0.03751605524409954


### Usando `cross_validate`
A principal diferença para o método anterior é que podemos setar várias métricas. Além disso, ele sempre retorna o tempo de `fit` e de `score`

In [26]:
nome_metricas = ['accuracy', 'precision_macro', 'recall_macro']
metricas = cross_validate(knn_pipeline, X, y, cv=5, scoring=nome_metricas)
for met in metricas:
    print(f"- {met}:")
    print(f"-- {metricas[met]}")
    print(f"-- {np.mean(metricas[met])} +- {np.std(metricas[met])}\n")    

- fit_time:
-- [0.00129795 0.00153828 0.00146532 0.00215125 0.00225377]
-- 0.001741313934326172 +- 0.0003859087222483761

- score_time:
-- [0.00671172 0.00605178 0.00701642 0.00863791 0.00928473]
-- 0.007540512084960938 +- 0.0012185628426625607

- test_accuracy:
-- [0.91666667 0.94444444 1.         1.         0.91428571]
-- 0.9550793650793651 +- 0.03817980032602645

- test_precision_macro:
-- [0.91538462 0.94405594 1.         1.         0.91538462]
-- 0.9549650349650349 +- 0.03823224723760816

- test_recall_macro:
-- [0.91904762 0.95238095 1.         1.         0.93333333]
-- 0.9609523809523809 +- 0.03359084207342554



## Gerando predições via `cross_val_predict`
Nesse caso, cada predição será obtida para o conjunto de teste de cada uma das *folders*. Em outras palavras, se `cv=5`, por exemplo, o modelo vai ser treinado para 4 partições e testado em 1, que gera as predições. Ao final das 5 execuções, os resultados são concatenados e retornados.

Podemos setar o parametro `method` para escolher qual predição será retornada.

**Observação**: os resultados para os dois métodos a seguir podem ser diferentes pois o modelo será retreinado para cada partição (que pode ser diferente). Vamos lidar com isso na sequência usando o `KFold`-like.

In [31]:
pred = cross_val_predict(knn_pipeline, X, y, cv=5)
print(pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 1 1 0 1 0
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 0 1 0 1 1 1 1 1 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [34]:
pred_prob = cross_val_predict(knn_pipeline, X, y, cv=5, method="predict_proba")
print(pred_prob[0])

[1. 0. 0.]


## Obtendo mais controle utilizando `KFold` e  `StratifiedKFold` 
Ambas as classes retornam um *generator* com os indices para selecionar os dados de acordo com o número de *folders* informadas. A principal diferença é que o `StratifiedKFold` estratifica os dados de acordo com as classes. Em outras palavras, ele balanceia a quantidade de amostras de cada classe entre as *folders*. Isso é relevante se o dataset é desbalanceado, algo comum em ML

#### `KFold`

In [36]:
n_folders = 5
cross_val = KFold(n_splits=n_folders, shuffle=True, random_state=32)
dados_cv = {f"folder_{f+1}": {"treino": None, "teste": None} for f in range(n_folders)}
k = 1
for indices_treino, indices_teste in cross_val.split(X):
    dados_cv[f"folder_{k}"]["treino"] = (X[indices_treino], y[indices_treino])
    dados_cv[f"folder_{k}"]["teste"] = (X[indices_teste], y[indices_teste])
    k+=1
print(dados_cv.keys())

dict_keys(['folder_1', 'folder_2', 'folder_3', 'folder_4', 'folder_5'])


As funções `cross_val_predict`, `cross_validate` e `cross_val_predict` aceitam que o parâmetro `cv` seja uma instancia da classe `KFold`-like por exemplo. Isso garante que as partições sejam exatamente as mesmas e evita o problema citado quando usamos `method="predict_proba"`

In [37]:
pred = cross_val_predict(knn_pipeline, X, y, cv=cross_val)
pred_prob = cross_val_predict(knn_pipeline, X, y, cv=cross_val, method="predict_proba")
print(pred[112])
print(pred_prob[112])

1
[0. 1. 0.]


Também podemos tomar controle do loop de execução do modelo. A gente iniciou esse notebook executando o modelo`n` vezes para partições diferentes de treino e teste. Podemos fazer o mesmo, mas agora usando as *folders*. Isso é que as funções anteriores fazem por trás do pano. Mas por algum motivo, você pode querer controlar esse loop.

In [38]:
acuracias = list()
for folder in dados_cv:       
    knn_pipeline.fit(dados_cv[folder]["treino"][0], dados_cv[folder]["treino"][1])
    ac_i = knn_pipeline.score(dados_cv[folder]["teste"][0], dados_cv[folder]["teste"][1])
    acuracias.append(ac_i)
    print(f"{folder}: {ac_i}")

print("\n- Acurácia das folders:")
print(f"Media: {round(np.mean(acuracias) * 100, 2)}%")
print(f"Desvio padrão: {round(np.std(acuracias) * 100, 2)}%")

folder_1: 0.9722222222222222
folder_2: 0.9722222222222222
folder_3: 0.9166666666666666
folder_4: 0.9142857142857143
folder_5: 1.0

- Acurácia das folders:
Media: 95.51%
Desvio padrão: 3.39%


#### `StratifiedKFold`
A ideia é basicamente a mesma da `KFold`, mas como vai ser estratificado por classe, precisamos passar `y` como parâmetro. Você pode substituir o *generator* obtido aqui para nas células anteriores

In [39]:
n_folders = 5
cross_val_strat = StratifiedKFold(n_splits=n_folders, shuffle=True, random_state=32)
dados_cv_strat = {f"folder_{f+1}": {"treino": None, "teste": None} for f in range(n_folders)}
k = 1
for indices_treino, indices_teste in cross_val_strat.split(X, y):
    dados_cv[f"folder_{k}"]["treino"] = (X[indices_treino], y[indices_treino])
    dados_cv[f"folder_{k}"]["teste"] = (X[indices_teste], y[indices_teste])
    k+=1
print(dados_cv.keys())

dict_keys(['folder_1', 'folder_2', 'folder_3', 'folder_4', 'folder_5'])


In [40]:
dados_cv["folder_1"]

{'treino': (array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
          1.065e+03],
         [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
          1.050e+03],
         [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
          1.185e+03],
         ...,
         [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
          8.350e+02],
         [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
          8.400e+02],
         [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
          5.600e+02]]),
  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,